<a href="https://colab.research.google.com/github/trituenhantaoio/DeepLearning-Tutorial/blob/master/M%E1%BA%A1ng_n%C6%A1_ron_trong_Pytorch_trituenhantao_io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Mạng nơ ron
===============

Mạng nơ ron có thể được xây dựng thông qua gói ``torch.nn``.

Trong bài trước, bạn đã học về ``autograd``, ``torch.nn`` dựa trên
``autograd`` để định nghĩa mô hình và tính đạo hàm.

Ví dụ, mạng nơ ron dưới đây phân loại các chữ số:

<img src='https://raw.githubusercontent.com/trituenhantaoio/DeepLearning-Tutorial/master/figures/mnist.png' alt='convnet'/>

Mạng nơ ron này nhận đầu vào ``input``, truyền nó qua các tầng và đưa ra kết quả ``output``.

Một quá trình chung để huấn luyện mạng nơ ron như sau:

- Định nghĩa một mạng nơ ron với các tham số (hoặc trọng số) có thể được học trong quá trình huấn luyện.
- Lặp qua một dataset chứa các ``input``
- Xử lý ``input`` qua mạng nơ ron
- Tính giá trị ``loss`` (``output`` sai khác bao nhiêu so với giá trị đúng)
- Lan truyền ngược độ dốc để cập nhật các trọng số
- Cập nhật trọng số của mạng với quy tắc đơn giản:
  ``weight = weight - learning_rate * gradient``

Định nghĩa mạng nơ ron
------------------

Chúng ta sẽ bắt đầu định nghĩa mạng nơ ron



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Chúng ta vừa định nghĩa hàm ``forward``. Hàm ``backward``
chứa các giá trị đạo hàm sẽ được tự động tính với ``autograd``.
Chúng ta có thể dùng bất kỳ phép toán trên Tensor nào với hàm ``forward``.

Các tham số của mạng có thể được trả về thông qua ``net.parameters()``



In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


Hãy thử với ``input`` ngẫu nhiên dạng 32x32.
Chú ý: kích thước ``input`` cần thiết của mạng này (LeNet) là 32x32. Để có thể sử dụng mạng này trên dataset MNIST, ta cần định hình lại các tấm ảnh về 32x32.



In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0119,  0.0622,  0.0178,  0.0636, -0.0043, -0.1265, -0.0876, -0.0235,
          0.1200,  0.0494]], grad_fn=<AddmmBackward>)


Thiết lập lại bộ đệm gradient của các trọng số về 0 và tính lan truyền ngược với giá trị gradient ngẫu nhiên:



In [ ]:
net.zero_grad()
out.backward(torch.randn(1, 10))

Toàn bộ gói ``torch.nn`` chỉ hỗ trợ ``input`` dưới dạng mini-batch chứ không hỗ trợ một mẫu riêng lẻ.

    Ví dụ, ``nn.Conv2d`` sẽ nhận vào một Tensor 4D với các chiều là
    ``nSamples x nChannels x Height x Width``.

    Nếu ta chỉ có một mẫu thì sử dụng ``input.unsqueeze(0)`` để tạo ra một chiều giả.

Trước khi học tiếp, hãy ôn lại một chút các bài đã học.

**Ôn tập:**
  -  ``torch.Tensor`` - Một *mảng đa chiều* hỗ trợ phép toán autograd
     như ``backward()``.
  -  ``nn.Module`` - Cấu phần giúp tạo mạng nơ ron *Một cách đơn giản để gói các trọng số*, hỗ trợ đưa lên GPU và các hoạt động liên quan.
  -  ``nn.Parameter`` - Một loại Tensor, được coi là tham số khi trở thành một thuộc tính của một ``Module``.
  -  ``autograd.Function`` - Định nghĩa phương thức lan truyền và lan truyền ngược một cách tự động. Mọi phương thức trên ``Tensor`` sẽ tạo ra một nút ``Function`` kết nối ``Tensor`` với hàm tạo ra nó và lưu trữ lịch sử tính toán.

**Như vậy chúng ta đã có thể:**
  -  Định nghĩa mạng nơ ron
  -  Xử lý luồng thông tin từ ``input`` và tiến hành lan truyền ngược.

**Các kiến thức mới:**
  -  Tính toán ``loss``
  -  Cập nhật trọng số mạng

Hàm Loss
-------------
Hàm loss nhận đầu vào là cặp (output, target), và tính xem khoảng cách giữa ``output`` và ``target`` là bao xa.

Có một số các
[hàm loss](https://pytorch.org/docs/nn.html#loss-functions) trong gói ``nn``.
Hàm loss đơn giản: ``nn.MSELoss`` sẽ tính sai số toàn phương trung bình giữa chúng.

Ví dụ:



In [ ]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9714, grad_fn=<MseLossBackward>)


Nếu theo dõi biến ``loss``, ta có thể thấy một biểu đồ tính toán như sau:


    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

Khi ta gọi ``loss.backward()``, toàn bộ biểu đồ được tính đạo hàm, tức là ``loss`` và các Tensor trong biểu đồ có thuộc tính ``requires_grad=True``.

Một cách trực quan hãy cùng theo dõi một số bước lan truyền ngược:



In [ ]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

Lan truyền ngược
--------
Để lan truyền ngược lỗi, tất cả những gì chúng ta phải làm là gọi ``loss.backward()``.
Mặc dù vậy, chúng ta cần xóa các giá trị gradient đã có để tránh việc gradient bị cộng dồn.


Chúng ta sẽ gọi ``loss.backward()``, và xem giá trị bias của conv1 trước và sau khi gọi phương thức này.



In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0108,  0.0025, -0.0007, -0.0046, -0.0062,  0.0010])


Bây giờ chúng ta sẽ xem cách sử dụng hàm loss.

**Đọc thêm:**

  Gói về mạng nơ ron chứa nhiều cấu phần và các hàm loss khác nhau giúp tạo nên các khối của mạng học sâu. Bạn có thể đọc đầy đủ tài liệu tại [đây](https://pytorch.org/docs/nn).

**Điều cuối cùng cần học trong bài:**

  - Cập nhật trọng số của mạng

Cập nhật trọng số
------------------
Quy tắc cập nhật đơn giản nhất là sử dụng Stochastic Gradient
Descent (SGD):

     weight = weight - learning_rate * gradient

Chúng ta có thể sử dụng Python code như sau:

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

Mặc dù vậy, đối với mạng nơ ron, chúng ta có thể sử dụng nhiều phương pháp cập nhật trọng số khác nhau như SGD, Nesterov-SGD, Adam, RMSProp, v..v..
Để làm được điều này, chúng ta sử dụng một gói mang tên ``torch.optim`` cài đặt các phương pháp trên. Cách sử dụng rất đơn giản:



In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

Lưu ý:

Chúng ta có thể thấy ``optimizer.zero_grad()`` cần được sử dụng trong quá trình cập nhật trọng số. Điều này là vì độ dốc được cộng dồn như đã giải thích trong phần `Lan truyền ngược`.
